In [1]:
import sys, os
from datetime import datetime, timedelta

project_root = os.path.abspath("..")
sys.path.insert(0, project_root)

data_dir = os.path.join(project_root, "data")
os.makedirs(data_dir, exist_ok=True)

print("project_root:", project_root)
print("data_dir:", data_dir)


project_root: /Users/mathisvillaret/Documents (mac)/Le CODE
data_dir: /Users/mathisvillaret/Documents (mac)/Le CODE/data


In [2]:
from import_other_options import import_sp500_options_data

start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=30)).date()

spx_df = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)

print("✓ spx_df loaded:", len(spx_df))
spx_df.head()


⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251212_005018.csv
📊 Total rows in CSV: 15197
✅ Loaded 6071 options after filtering by date range (2025-12-15 to 2026-01-14)
   (9126 options filtered out)
✓ spx_df loaded: 6071


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,best_bid_price,best_ask_price,volume,instrument_name,contractSymbol,underlying_ticker
503,2025-12-15,2025-12-15,2800.0,P,2.070317,0.22,6901.0,291.0,0.0,0.05,0.0,0.05,251.0,^SPX-2025-12-15-2800.0-P,SPXW251215P02800000,^SPX
504,2025-12-15,2025-12-15,3000.0,P,1.921875,0.35,6901.0,5.0,0.0,0.05,0.0,0.05,NaN,^SPX-2025-12-15-3000.0-P,SPXW251215P03000000,^SPX
505,2025-12-15,2025-12-15,3200.0,P,1.773439,0.10,6901.0,37.0,0.0,0.05,0.0,0.05,30.0,^SPX-2025-12-15-3200.0-P,SPXW251215P03200000,^SPX
506,2025-12-15,2025-12-15,3400.0,P,1.640627,0.05,6901.0,256.0,0.0,0.05,0.0,0.05,2.0,^SPX-2025-12-15-3400.0-P,SPXW251215P03400000,^SPX
507,2025-12-15,2025-12-15,3600.0,P,1.515627,0.05,6901.0,12.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-15-3600.0-P,SPXW251215P03600000,^SPX


In [3]:
from iv_surface_spx import SPXIVSurface, SurfaceConfig

cfg = SurfaceConfig(
    r=0.05,
    min_bid=0.01,
    max_rel_spread=0.25,
    min_oi=10,
    min_volume=1,
    grid_n=60,
    rbf_smoothing=0.5
)

spx_surface = SPXIVSurface(spx_df, cfg)
print("Rows after cleaning:", len(spx_surface.df))
spx_surface.df.head()


Rows after cleaning: 2000


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,...,underlying_ticker,T,S,bid,ask,mid,rel_spread,iv_pct,F,x
0,2025-12-15,2025-12-15,6760.0,P,0.098489,1.50,6901.0,238,0.9,1.15,...,^SPX,0.000001,6901.0,0.9,1.15,1.025,0.243902,9.848924,6920.550002,-0.023472
1,2025-12-15,2025-12-15,6765.0,P,0.097085,1.60,6901.0,172,1.0,1.25,...,^SPX,0.000001,6901.0,1.0,1.25,1.125,0.222222,9.708545,6920.550002,-0.022733
2,2025-12-15,2025-12-15,6770.0,P,0.095529,1.67,6901.0,652,1.1,1.35,...,^SPX,0.000001,6901.0,1.1,1.35,1.225,0.204082,9.552906,6920.550002,-0.021994
3,2025-12-15,2025-12-15,6775.0,P,0.093851,2.00,6901.0,770,1.2,1.45,...,^SPX,0.000001,6901.0,1.2,1.45,1.325,0.188679,9.385061,6920.550002,-0.021256
4,2025-12-15,2025-12-15,6780.0,P,0.092645,1.98,6901.0,906,1.3,1.60,...,^SPX,0.000001,6901.0,1.3,1.60,1.450,0.206897,9.264518,6920.550002,-0.020518


In [4]:
fig = spx_surface.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()


### Summary (SPX IV Surface — OTM, forward-moneyness)

- The surface exhibits a **strong downside skew**: implied volatility increases sharply as  
  **log-moneyness** $\ln(K/F)$ becomes more negative (deep OTM puts).  
  This is typical for SPX and reflects **crash-risk insurance demand**.

- Volatility varies with **time to expiry**, and the **term structure depends on moneyness**  
  (the maturity effect is not uniform across strikes).

- The “striped” pattern in the white points is expected because option quotes exist on  
  **discrete expiries and strikes**; the smooth surface is an interpolation across that grid.

- Extremely high IV levels (e.g., **60–70%**) in the far left tail are likely driven by  
  **illiquid/unstable quotes and interpolation extrapolation**, so the outer tail should be  
  treated cautiously (e.g., **outlier filtering** or **restricting the moneyness range** before fitting).


In [5]:
from import_other_options import import_sp500_options_data

start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=30)).date()

spx_df1 = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)

print("✓ spx_df loaded:", len(spx_df1))
spx_df


⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251212_005018.csv
📊 Total rows in CSV: 15197
✅ Loaded 6071 options after filtering by date range (2025-12-15 to 2026-01-14)
   (9126 options filtered out)
✓ spx_df loaded: 6071


,expiry_str,expiry_date,strike,type,mark_iv,mark_price,underlying_price,open_interest,bid_price,ask_price,best_bid_price,best_ask_price,volume,instrument_name,contractSymbol,underlying_ticker
503,2025-12-15,2025-12-15,2800.0,P,2.070317,0.22,6901.0,291.0,0.0,0.05,0.0,0.05,251.0,^SPX-2025-12-15-2800.0-P,SPXW251215P02800000,^SPX
504,2025-12-15,2025-12-15,3000.0,P,1.921875,0.35,6901.0,5.0,0.0,0.05,0.0,0.05,NaN,^SPX-2025-12-15-3000.0-P,SPXW251215P03000000,^SPX
505,2025-12-15,2025-12-15,3200.0,P,1.773439,0.10,6901.0,37.0,0.0,0.05,0.0,0.05,30.0,^SPX-2025-12-15-3200.0-P,SPXW251215P03200000,^SPX
506,2025-12-15,2025-12-15,3400.0,P,1.640627,0.05,6901.0,256.0,0.0,0.05,0.0,0.05,2.0,^SPX-2025-12-15-3400.0-P,SPXW251215P03400000,^SPX
507,2025-12-15,2025-12-15,3600.0,P,1.515627,0.05,6901.0,12.0,0.0,0.05,0.0,0.05,1.0,^SPX-2025-12-15-3600.0-P,SPXW251215P03600000,^SPX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6569,2026-01-14,2026-01-14,7050.0,C,0.101453,42.80,6901.0,NaN,31.8,32.40,31.8,32.40,NaN,^SPX-2026-01-14-7050.0-C,SPXW260114C07050000,^SPX
6570,2026-01-14,2026-01-14,7075.0,C,0.099946,37.08,6901.0,NaN,NaN,25.80,NaN,25.80,NaN,^SPX-2026-01-14-7075.0-C,SPXW260114C07075000,^SPX
6571,2026-01-14,2026-01-14,7100.0,C,0.099527,27.64,6901.0,NaN,20.6,20.80,20.6,20.80,NaN,^SPX-2026-01-14-7100.0-C,SPXW260114C07100000,^SPX
6572,2026-01-14,2026-01-14,7125.0,C,0.099332,22.60,6901.0,NaN,16.4,16.70,16.4,16.70,NaN,^SPX-2026-01-14-7125.0-C,SPXW260114C07125000,^SPX


In [6]:
cfg = SurfaceConfig(
    min_T=1/365,  # Min 3-4 days Base 1 year =1, 1 month =1/12 etc.
    max_T=20   #
)
spx_surface_test = SPXIVSurface(spx_df1, cfg)
surface = SPXIVSurface(spx_df, cfg=cfg)

fig = spx_surface_test.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()


In [7]:
print(f"Options bf filtering : {len(spx_df1)}")
surface = SPXIVSurface(spx_df1, cfg=cfg)
print(f"Options after filtering: {len(surface.df)}")

Options bf filtering : 6071
Options after filtering: 1881


In [8]:
from datetime import datetime
import pandas as pd

# Convert expiry dates to datetime
spx_df['expiry_date'] = pd.to_datetime(spx_df['expiry_date'])

# Set reference date (as of today)
as_of = datetime.now()

# Calculate time to expiration in years
spx_df['T'] = (spx_df['expiry_date'] - pd.Timestamp(as_of)).dt.total_seconds() / (365.25 * 24 * 3600)

# Display the time range in your data
print(f"T min in your data: {spx_df['T'].min():.3f}")
print(f"T max in your data: {spx_df['T'].max():.3f}")

T min in your data: -0.003
T max in your data: 0.079


In [9]:
# Test avec un intervalle plus court
start_date = datetime.now().date()
end_date = (datetime.now() + timedelta(days=1000)).date()  # Seulement 30 jours

spx_df_short = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir
)
print(f"Options avec intervalle de 30 jours: {len(spx_df_short)}")
# Devrait être moins que 14694

⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251212_005018.csv
📊 Total rows in CSV: 15197
✅ Loaded 14694 options after filtering by date range (2025-12-15 to 2028-09-10)
   (503 options filtered out)
Options avec intervalle de 30 jours: 14694


In [14]:
spx_df2 = import_sp500_options_data(
    start_date=start_date,
    end_date=end_date,
    ticker="^SPX",
    output_dir=data_dir,
    filter_by_date=False  # ⬅️ Désactive le filtrage par date
)

⚠️  Warning: Data directory path doesn't contain 'Option_Pricing': /Users/mathisvillaret/Documents (mac)/Le CODE/data
   Expected path should contain: .../Option_Pricing/data/
   Current working directory: /Users/mathisvillaret/Documents (mac)/Le CODE/Option_Pricing
   ✅ Fixed path to: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📁 Data will be saved to / loaded from: /Users/mathisvillaret/Documents (mac)/Le CODE/data
📂 Existing CSV found, loading instead of fetching from Yahoo Finance:
   /Users/mathisvillaret/Documents (mac)/Le CODE/data/sp500_options_SPX_20251212_005018.csv
📊 Total rows in CSV: 15197
✅ Loaded all 15197 options from CSV (no date filtering)


In [15]:
cfg = SurfaceConfig(
    min_T=1/365,  # Min 3-4 days Base 1 year =1, 1 month =1/12 etc.
    max_T=2   #
)
spx_surface_test = SPXIVSurface(spx_df2, cfg)
surface = SPXIVSurface(spx_df2, cfg=cfg)

fig = spx_surface_test.plot(
    title="SPX Implied Volatility Surface (OTM, forward-moneyness)",
    interpolate=True
)
fig.show()
